# Finding corrupted epochs (for QS)

1. Find corrupted epochs by plotting and looking at spectrograms.
2. Find start epoch and end epoch of data corruption.
3. Mark those as Artifcat (-1)
4. Generate spectro-hypno plots for them

In [1]:
import numpy as np
import mne
from scipy import signal
from scipy.interpolate import RectBivariateSpline
from mne.filter import resample, filter_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lspopt import spectrogram_lspopt
from matplotlib.colors import Normalize, ListedColormap

import logging
LOGGING_TYPES = dict(DEBUG=logging.DEBUG, INFO=logging.INFO, WARNING=logging.WARNING,
                     ERROR=logging.ERROR, CRITICAL=logging.CRITICAL)
logger = logging.getLogger('yasa')

%matplotlib qt

## Find corrupted epochs by plotting and looking at spectrograms

In [16]:
# Load the EDF file
fname = "47"  # define here
lr = "R"  # define here
location = f"/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/{fname}"
raw = mne.io.read_raw_edf(f"{location}/EEG {lr}.edf", preload=True, verbose=0)
raw.pick_types(eeg=True)

mne.viz.plot_raw(raw, use_opengl=False)

Opening raw-browser...


Closing raw-browser...
Channels marked as bad:
none
Closing raw-browser...
Channels marked as bad:
none


## Mark corrupted epochs as Artifcat (-1)

In [13]:
location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos"

epoch_f = lambda h : h * 60 * 2

START = epoch_f(2.4)
END   = epoch_f(2.8)

replacement = ""
cnt = 1

with open(f"{location_hypno}/session_{fname}_synced.txt", "r") as f:
    for line in f:
        if START <= cnt <= END:
            replacement = replacement + "-1\t0" + "\n"
        else:
            replacement += line

        cnt += 1

with open(f"{location_hypno}/session_{fname}_synced_updated.txt", "w") as f:
    f.write(replacement)


## Generate spectro-hypno plots for them

In [41]:
def plot_spectrogram(
    data, sf, hypno=None, win_sec=30, fmin=0.5, fmax=25, trimperc=2.5, cmap="RdBu_r"
):
    """
    Plot a full-night multi-taper spectrogram, optionally with the hypnogram on top.
    For more details, please refer to the `Jupyter notebook
    <https://github.com/raphaelvallat/yasa/blob/master/notebooks/10_spectrogram.ipynb>`_
    .. versionadded:: 0.1.8
    Parameters
    ----------
    data : :py:class:`numpy.ndarray`
        Single-channel EEG data. Must be a 1D NumPy array.
    sf : float
        The sampling frequency of data AND the hypnogram.
    hypno : array_like
        Sleep stage (hypnogram), optional.
        The hypnogram must have the exact same number of samples as ``data``.
        To upsample your hypnogram, please refer to :py:func:`yasa.hypno_upsample_to_data`.
        .. note::
            The default hypnogram format in YASA is a 1D integer
            vector where:
            - -2 = Unscored
            - -1 = Artefact / Movement
            - 0 = Wake
            - 1 = N1 sleep
            - 2 = N2 sleep
            - 3 = N3 sleep
            - 4 = REM sleep
    win_sec : int or float
        The length of the sliding window, in seconds, used for multitaper PSD
        calculation. Default is 30 seconds. Note that ``data`` must be at least
        twice longer than ``win_sec`` (e.g. 60 seconds).
    fmin, fmax : int or float
        The lower and upper frequency of the spectrogram. Default 0.5 to 25 Hz.
    trimperc : int or float
        The amount of data to trim on both ends of the distribution when
        normalizing the colormap. This parameter directly impacts the
        contrast of the spectrogram plot (higher values = higher contrast).
        Default is 2.5, meaning that the min and max of the colormap
        are defined as the 2.5 and 97.5 percentiles of the spectrogram.
    cmap : str
        Colormap. Default to 'RdBu_r'.
    Returns
    -------
    fig : :py:class:`matplotlib.figure.Figure`
        Matplotlib Figure
    Examples
    --------
    1. Full-night multitaper spectrogram on Cz, no hypnogram
    .. plot::
        >>> import yasa
        >>> import numpy as np
        >>> # In the next 5 lines, we're loading the data from GitHub.
        >>> import requests
        >>> from io import BytesIO
        >>> r = requests.get('https://github.com/raphaelvallat/yasa/raw/master/notebooks/data_full_6hrs_100Hz_Cz%2BFz%2BPz.npz', stream=True)
        >>> npz = np.load(BytesIO(r.raw.read()))
        >>> data = npz.get('data')[0, :]
        >>> sf = 100
        >>> fig = yasa.plot_spectrogram(data, sf)
    2. Full-night multitaper spectrogram on Cz with the hypnogram on top
    .. plot::
        >>> import yasa
        >>> import numpy as np
        >>> # In the next lines, we're loading the data from GitHub.
        >>> import requests
        >>> from io import BytesIO
        >>> r = requests.get('https://github.com/raphaelvallat/yasa/raw/master/notebooks/data_full_6hrs_100Hz_Cz%2BFz%2BPz.npz', stream=True)
        >>> npz = np.load(BytesIO(r.raw.read()))
        >>> data = npz.get('data')[0, :]
        >>> sf = 100
        >>> # Load the 30-sec hypnogram and upsample to data
        >>> hypno = np.loadtxt('https://raw.githubusercontent.com/raphaelvallat/yasa/master/notebooks/data_full_6hrs_100Hz_hypno_30s.txt')
        >>> hypno = yasa.hypno_upsample_to_data(hypno, 1/30, data, sf)
        >>> fig = yasa.plot_spectrogram(data, sf, hypno, cmap='Spectral_r')
    """
    # Increase font size while preserving original
    old_fontsize = plt.rcParams["font.size"]
    plt.rcParams.update({"font.size": 18})

    # Safety checks
    assert isinstance(data, np.ndarray), "Data must be a 1D NumPy array."
    assert isinstance(sf, (int, float)), "sf must be int or float."
    assert data.ndim == 1, "Data must be a 1D (single-channel) NumPy array."
    assert isinstance(win_sec, (int, float)), "win_sec must be int or float."
    assert isinstance(fmin, (int, float)), "fmin must be int or float."
    assert isinstance(fmax, (int, float)), "fmax must be int or float."
    assert fmin < fmax, "fmin must be strictly inferior to fmax."
    assert fmax < sf / 2, "fmax must be less than Nyquist (sf / 2)."

    # Calculate multi-taper spectrogram
    nperseg = int(win_sec * sf)
    assert data.size > 2 * nperseg, "Data length must be at least 2 * win_sec."
    f, t, Sxx = spectrogram_lspopt(data, sf, nperseg=nperseg, noverlap=0)
    Sxx = 10 * np.log10(Sxx)  # Convert uV^2 / Hz --> dB / Hz

    # Select only relevant frequencies (up to 30 Hz)
    good_freqs = np.logical_and(f >= fmin, f <= fmax)
    Sxx = Sxx[good_freqs, :]
    f = f[good_freqs]
    t /= 3600  # Convert t to hours

    # Normalization
    vmin, vmax = np.percentile(Sxx, [0 + trimperc, 100 - trimperc])
    norm = Normalize(vmin=vmin, vmax=vmax)

    if hypno is None:
        fig, ax = plt.subplots(nrows=1, figsize=(12, 4))
        im = ax.pcolormesh(
            t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto"
        )
        ax.set_xlim(0, t.max())
        ax.set_ylabel("Frequency [Hz]")
        ax.set_xlabel("Time [hrs]")

        # Add colorbar
        cbar = fig.colorbar(im, ax=ax, shrink=0.95, fraction=0.1, aspect=25)
        cbar.ax.set_ylabel("Log Power (dB / Hz)", rotation=270, labelpad=20)
        return fig
    else:
        hypno = np.asarray(hypno).astype(int)
        assert hypno.ndim == 1, "Hypno must be 1D."
        assert hypno.size == data.size, "Hypno must have the same sf as data."
        t_hyp = np.arange(hypno.size) / (sf * 3600)
        # Make sure that REM is displayed after Wake
        hypno = (
            pd.Series(hypno).map({-2: -2, -1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 5: 1}).values
        )
        hypno_rem = np.ma.masked_not_equal(hypno, 1)

        fig, (ax0, ax1) = plt.subplots(
            nrows=2, figsize=(12, 6), gridspec_kw={"height_ratios": [1, 2]}
        )
        plt.subplots_adjust(hspace=0.1)

        # Hypnogram (top axis)
        ax0.step(t_hyp, -1 * hypno, color="k")
        ax0.step(t_hyp, -1 * hypno_rem, color="r")
        if -2 in hypno and -1 in hypno:
            # Both Unscored and Artefacts are present
            ax0.set_yticks([2, 1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)
        elif -2 in hypno and -1 not in hypno:
            # Only Unscored are present
            ax0.set_yticks([2, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Uns", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 2.5)

        elif -2 not in hypno and -1 in hypno:
            # Only Artefacts are present
            ax0.set_yticks([1, 0, -1, -2, -3, -4])
            ax0.set_yticklabels(["Art", "W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 1.5)
        else:
            # No artefacts or Unscored
            ax0.set_yticks([0, -1, -2, -3, -4])
            ax0.set_yticklabels(["W", "R", "N1", "N2", "N3"])
            ax0.set_ylim(-4.5, 0.5)
        ax0.set_xlim(0, t_hyp.max())
        ax0.set_ylabel("Stage")
        ax0.xaxis.set_visible(False)
        ax0.spines["right"].set_visible(False)
        ax0.spines["top"].set_visible(False)

        # Spectrogram (bottom axis)
        im = ax1.pcolormesh(
            t, f, Sxx, norm=norm, cmap=cmap, antialiased=True, shading="auto"
        )
        ax1.set_xlim(0, t.max())
        ax1.set_ylabel("Frequency [Hz]")
        ax1.set_xlabel("Time [hrs]")

        # Revert font-size
        plt.rcParams.update({"font.size": old_fontsize})
        return fig


def format_seconds_to_hhmmss(seconds):
    hours = seconds // (60 * 60)
    seconds %= 60 * 60
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i:%02i" % (hours, minutes, seconds)


def set_log_level(verbose=None):
    """Convenience function for setting the logging level.
    This function comes from the PySurfer package. See :
    https://github.com/nipy/PySurfer/blob/master/surfer/utils.py
    Parameters
    ----------
    verbose : bool, str, int, or None
        The verbosity of messages to print. If a str, it can be either
        PROFILER, DEBUG, INFO, WARNING, ERROR, or CRITICAL.
    """
    logger = logging.getLogger("yasa")
    if isinstance(verbose, bool):
        verbose = "INFO" if verbose else "WARNING"
    if isinstance(verbose, str):
        if verbose.upper() in LOGGING_TYPES:
            verbose = verbose.upper()
            verbose = LOGGING_TYPES[verbose]
            logger.setLevel(verbose)
        else:
            raise ValueError("verbose must be in %s" % ", ".join(LOGGING_TYPES))


def hypno_upsample_to_data(hypno, sf_hypno, data, sf_data=None, verbose=True):
    """Upsample an hypnogram to a given sampling frequency and fit the
    resulting hypnogram to corresponding EEG data, such that the hypnogram
    and EEG data have the exact same number of samples.
    .. versionadded:: 0.1.5
    Parameters
    ----------
    hypno : array_like
        The sleep staging (hypnogram) 1D array.
    sf_hypno : float
        The current sampling frequency of the hypnogram, in Hz, e.g.
        * 1/30 = 1 value per each 30 seconds of EEG data,
        * 1 = 1 value per second of EEG data
    data : array_like or :py:class:`mne.io.BaseRaw`
        1D or 2D EEG data. Can also be a :py:class:`mne.io.BaseRaw`, in which
        case ``data`` and ``sf_data`` will be automatically extracted.
    sf_data : float
        The sampling frequency of ``data``, in Hz (e.g. 100 Hz, 256 Hz, ...).
        Can be omitted if ``data`` is a :py:class:`mne.io.BaseRaw`.
    verbose : bool or str
        Verbose level. Default (False) will only print warning and error
        messages. The logging levels are 'debug', 'info', 'warning', 'error',
        and 'critical'. For most users the choice is between 'info'
        (or ``verbose=True``) and warning (``verbose=False``).
    Returns
    -------
    hypno : array_like
        The hypnogram, upsampled to ``sf_data`` and cropped/padded to ``max(data.shape)``.
    Warns
    -----
    UserWarning
        If the upsampled ``hypno`` is shorter / longer than ``max(data.shape)``
        and therefore needs to be padded/cropped respectively. This output can be disabled by
        passing ``verbose='ERROR'``.
    """
    set_log_level(verbose)
    if isinstance(data, mne.io.BaseRaw):
        sf_data = data.info["sfreq"]
        data = data.times

    # Upsample the hypnogram to a given sampling frequency
    repeats = sf_data / sf_hypno
    assert sf_hypno <= sf_data, "sf_hypno must be less than sf_data."
    assert repeats.is_integer(), "sf_hypno / sf_data must be a whole number."
    assert isinstance(hypno, (list, np.ndarray, pd.Series))
    hypno_up = np.repeat(np.asarray(hypno), repeats)

    # Crop or pad the hypnogram to fit the length of data.
    # Check if data is an MNE raw object
    hypno = hypno_up
    sf = sf_data
    if isinstance(data, mne.io.BaseRaw):
        sf = data.info["sfreq"]
        data = data.times  # 1D array and does not require to preload data
    data = np.asarray(data)
    hypno = np.asarray(hypno)
    assert hypno.ndim == 1, "Hypno must be 1D."
    npts_hyp = hypno.size
    npts_data = max(data.shape)  # Support for 2D data
    if npts_hyp < npts_data:
        # Hypnogram is shorter than data
        npts_diff = npts_data - npts_hyp
        if sf is not None:
            dur_diff = npts_diff / sf
            logger.warning(
                "Hypnogram is SHORTER than data by %.2f seconds. "
                "Padding hypnogram with last value to match data.size." % dur_diff
            )
        else:
            logger.warning(
                "Hypnogram is SHORTER than data by %i samples. "
                "Padding hypnogram with last value to match data.size." % npts_diff
            )
        hypno = np.pad(hypno, (0, npts_diff), mode="edge")
    elif npts_hyp > npts_data:
        # Hypnogram is longer than data
        npts_diff = npts_hyp - npts_data
        if sf is not None:
            dur_diff = npts_diff / sf
            logger.warning(
                "Hypnogram is LONGER than data by %.2f seconds. "
                "Cropping hypnogram to match data.size." % dur_diff
            )
        else:
            logger.warning(
                "Hypnogram is LONGER than data by %i samples. "
                "Cropping hypnogram to match data.size." % npts_diff
            )
        hypno = hypno[0:npts_data]

    return hypno


def transition_matrix(hypno):
    """Create a state-transition matrix from an hypnogram.
    .. versionadded:: 0.1.9
    Parameters
    ----------
    hypno : array_like
        Hypnogram. The dtype of ``hypno`` must be integer
        (e.g. [0, 2, 2, 1, 1, 1, ...]). The sampling frequency must be the
        original one, i.e. 1 value per 30 seconds if the staging was done in
        30 seconds epochs. Using an upsampled hypnogram will result in an
        incorrect transition matrix.
        For best results, we recommend using an hypnogram cropped to
        either the time in bed (TIB) or the sleep period time (SPT), without
        any artefact / unscored epochs.
    Returns
    -------
    counts : :py:class:`pandas.DataFrame`
        Counts transition matrix (number of transitions from stage A to
        stage B). The pre-transition states are the rows and the
        post-transition states are the columns.
    probs : :py:class:`pandas.DataFrame`
        Conditional probability transition matrix, i.e.
        given that current state is A, what is the probability that
        the next state is B.
        ``probs`` is a `right stochastic matrix
        <https://en.wikipedia.org/wiki/Stochastic_matrix>`_,
        i.e. each row sums to 1.
    Examples
    --------
    >>> import numpy as np
    >>> from yasa import transition_matrix
    >>> a = [0, 0, 0, 1, 1, 0, 1, 2, 2, 3, 3, 2, 3, 3, 0, 2, 2, 1, 2, 2, 3, 3]
    >>> counts, probs = transition_matrix(a)
    >>> counts
           0  1  2  3
    Stage
    0      2  2  1  0
    1      1  1  2  0
    2      0  1  3  3
    3      1  0  1  3
    >>> probs.round(2)
              0     1     2     3
    Stage
    0      0.40  0.40  0.20  0.00
    1      0.25  0.25  0.50  0.00
    2      0.00  0.14  0.43  0.43
    3      0.20  0.00  0.20  0.60
    Several metrics of sleep fragmentation can be calculated from the
    probability matrix. For example, the stability of sleep stages can be
    calculated by taking the average of the diagonal values (excluding Wake
    and N1 sleep):
    >>> np.diag(probs.loc[2:, 2:]).mean().round(3)
    0.514
    Finally, we can plot the transition matrix using :py:func:`seaborn.heatmap`
    .. plot::
        >>> import numpy as np
        >>> import seaborn as sns
        >>> import matplotlib.pyplot as plt
        >>> from yasa import transition_matrix
        >>> # Calculate probability matrix
        >>> a = [1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 1, 1, 0, 0]
        >>> _, probs = transition_matrix(a)
        >>> # Start the plot
        >>> grid_kws = {"height_ratios": (.9, .05), "hspace": .1}
        >>> f, (ax, cbar_ax) = plt.subplots(2, gridspec_kw=grid_kws,
        ...                                 figsize=(5, 5))
        >>> sns.heatmap(probs, ax=ax, square=False, vmin=0, vmax=1, cbar=True,
        ...             cbar_ax=cbar_ax, cmap='YlOrRd', annot=True, fmt='.2f',
        ...             cbar_kws={"orientation": "horizontal", "fraction": 0.1,
        ...                       "label": "Transition probability"})
        >>> ax.set_xlabel("To sleep stage")
        >>> ax.xaxis.tick_top()
        >>> ax.set_ylabel("From sleep stage")
        >>> ax.xaxis.set_label_position('top')
    """
    x = np.asarray(hypno, dtype=int)
    unique, inverse = np.unique(x, return_inverse=True)  # unique is sorted
    n = unique.size
    # Integer transition counts
    counts = np.zeros((n, n), dtype=int)
    np.add.at(counts, (inverse[:-1], inverse[1:]), 1)
    # Conditional probabilities
    probs = counts / counts.sum(axis=-1, keepdims=True)
    # Convert to a Pandas DataFrame
    counts = pd.DataFrame(counts, index=unique, columns=unique)
    probs = pd.DataFrame(probs, index=unique, columns=unique)
    counts.index.name = "From Stage"
    probs.index.name = "From Stage"
    counts.columns.name = "To Stage"
    probs.columns.name = "To Stage"
    return counts, probs


In [15]:
import os

folder = "/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/"

sub_folders = [fname]

for folder in sub_folders:

    for LR in ["R", "L"]:

        location = f"/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/{folder}"
        raw = mne.io.read_raw_edf(f"{location}/EEG {LR}.edf", preload=True, verbose=0)
        raw.pick_types(eeg=True)
        # fig = raw.plot(use_opengl=False)

        # Apply a bandpass filter between 0.5 - 45 Hz
        raw.filter(0.5, 45)

        # Extract the data and convert from V to uV
        data = raw._data * 1e6
        sf = raw.info["sfreq"]
        chan = raw.ch_names

        # Let's have a look at the data
        print("Chan =", chan)
        print("Sampling frequency =", sf, "Hz")
        print("Data shape =", data.shape)

        print(
            f"Duration: {data.shape[1]/sf} (sec) OR {format_seconds_to_hhmmss(data.shape[1]/sf)}"
        )

        fig = plot_spectrogram(data[0], sf, fmax=45)
        plt.title(
            f"Spectrogram of {folder} - {format_seconds_to_hhmmss(data.shape[1]/sf)}",
            fontsize=16,
        )
        plt.tight_layout()
        plt.savefig(f"spectro QS {folder} {LR}.png", dpi=100, bbox_inches="tight")
        plt.show()

        location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos"
        hypno_30s = np.loadtxt(f"{location_hypno}/session_{folder}_synced_updated.txt")[:, 0]
        hypno = hypno_upsample_to_data(
            hypno=hypno_30s, sf_hypno=(1 / 30), data=data, sf_data=sf
        )

        fig = plot_spectrogram(data[0], sf, hypno=hypno, fmax=30, trimperc=5)
        fig.suptitle(
            f"Spectrogram and Hypnogram of QS {folder} ({LR}) - {format_seconds_to_hhmmss(data.shape[1]/sf)}",
            fontsize=16,
        )
        plt.tight_layout()
        plt.savefig(f"spectro-hypno QS {folder} {LR}.png", dpi=100, bbox_inches="tight")
        plt.show()


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Chan = ['EEG R']
Sampling frequency = 256.0 Hz
Data shape = (1, 5450240)
Duration: 21290.0 (sec) OR 05:54:50


Hypnogram is SHORTER than data by 20.00 seconds. Padding hypnogram with last value to match data.size.


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Chan = ['EEG L']
Sampling frequency = 256.0 Hz
Data shape = (1, 5450240)
Duration: 21290.0 (sec) OR 05:54:50


Hypnogram is SHORTER than data by 20.00 seconds. Padding hypnogram with last value to match data.size.


# All code in one cell

The code bellow works if you have mutiple clipping sections.

In [42]:
# Load the EDF file
fname = "28"  # define here
lr = "R"  # define here
location = f"/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/{fname}"
raw = mne.io.read_raw_edf(f"{location}/EEG {lr}.edf", preload=True, verbose=0)
raw.pick_types(eeg=True)
location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos"

epoch_f = lambda h: int(h * 60 * 2)

START = [epoch_f(1/120)] # 1/120
END = [epoch_f(0.05)]

replacement = ""
cnt = 1
replace = False

with open(f"{location_hypno}/session_{fname}_synced.txt", "r") as f:
    for line in f:
        for i in range(len(START)):
            if cnt == START[i]:
                replace = True
                break
            elif cnt == END[i] + 1:
                replace = False
                break

        if replace:
            replacement = replacement + "-1\t0" + "\n"
        else:
            replacement += line

        cnt += 1

with open(f"{location_hypno}/session_{fname}_synced_updated.txt", "w") as f:
    f.write(replacement)


import os

folder = "/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/"

sub_folders = [fname]

for folder in sub_folders:

    for LR in ["R", "L"]:

        location = f"/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/QS/{folder}"
        raw = mne.io.read_raw_edf(f"{location}/EEG {LR}.edf", preload=True, verbose=0)
        raw.pick_types(eeg=True)
        # fig = raw.plot(use_opengl=False)

        # Apply a bandpass filter between 0.5 - 45 Hz
        raw.filter(0.5, 45)

        # Extract the data and convert from V to uV
        data = raw._data * 1e6
        sf = raw.info["sfreq"]
        chan = raw.ch_names

        # Let's have a look at the data
        print("Chan =", chan)
        print("Sampling frequency =", sf, "Hz")
        print("Data shape =", data.shape)

        print(
            f"Duration: {data.shape[1]/sf} (sec) OR {format_seconds_to_hhmmss(data.shape[1]/sf)}"
        )

        fig = plot_spectrogram(data[0], sf, fmax=45)
        plt.title(
            f"Spectrogram of {folder} - {format_seconds_to_hhmmss(data.shape[1]/sf)}",
            fontsize=16,
        )
        plt.tight_layout()
        plt.savefig(f"spectro QS {folder} {LR}.png", dpi=100, bbox_inches="tight")
        plt.show()

        location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos"
        hypno_30s = np.loadtxt(f"{location_hypno}/session_{folder}_synced_updated.txt")[
            :, 0
        ]
        hypno = hypno_upsample_to_data(
            hypno=hypno_30s, sf_hypno=(1 / 30), data=data, sf_data=sf
        )

        fig = plot_spectrogram(data[0], sf, hypno=hypno, fmax=30, trimperc=5)
        fig.suptitle(
            f"Spectrogram and Hypnogram of QS {folder} ({LR}) - {format_seconds_to_hhmmss(data.shape[1]/sf)}",
            fontsize=16,
        )
        plt.tight_layout()
        plt.savefig(f"spectro-hypno QS {folder} {LR}.png", dpi=100, bbox_inches="tight")
        plt.show()


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Chan = ['EEG R']
Sampling frequency = 256.0 Hz
Data shape = (1, 5982720)
Duration: 23370.0 (sec) OR 06:29:30
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower 